In [ ]:
!pip install -q neo4j-driver
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.5/312.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase
import spacy
import gradio as gr

In [ ]:
driver = GraphDatabase.driver("bolt://3.230.115.146:7687",
                              auth=("neo4j",
                                    "powers-components-turbine"))

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None

In [ ]:
# def chatbot(input, history=[]):
#     output = get_answer(input)
#     history.append((input, output))
#     return history, history

def chatbot(input, history=None):
    if history is None:
        history = []
    output = get_answer(input)
    history.append((input, output))
    return history, history


In [ ]:
def get_answer(question):
    doc = nlp(question)
    movie_title = extract_entity(question, ["WORK_OF_ART", "MOVIE"])
    director_name = extract_entity(question, ["PERSON"])
    if movie_title is None:
        return "Sorry, I don't understand what movie you're asking about."

    with driver.session() as session:
        if "release" in question or "released" in question:
            result = session.run(f"MATCH (m:Movie) WHERE m.title = '{movie_title}' RETURN m.released")
            if result.peek() is None:
                return f"No information found for the movie '{movie_title}'"
            for record in result:
                return f"The release date of '{movie_title}' is: {record['m.released']}"
        else:
            return "Sorry, I don't understand what you're asking."

In [ ]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)

/usr/local/lib/python3.11/dist-packages/gradio/components/base.py:433: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  obj = utils.component_or_layout_class(cls_name)(render=render)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://81c25b4f6dd7cbc36e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://81c25b4f6dd7cbc36e.gradio.live


In [ ]:
## Rule based